# 9.1 Merging ticker changes
In section 1.7 we have acquired the list of renamings. Now we will use them to merge our data. We have to be aware that it is possible for a ticker to used multiple times, so the <code>ticker_changes.csv</code> may contain multiple of the same tickers in the 'from' and 'to' column. 

After processing the ticker changes we will create a <code>tickers_v5.csv</code> which will be our definitive ticker list. This contains a column 'tickers_old', which will containa list of (date_of_change, ticker) pairs. So if A changes to B on day 2, and B changes to C on day 5, tickers_old for D will contain [[2, A], [5, B]].

The process will be as follows:
* As long as we have ticker changes to process
    * Loop through <code>tickers_v4.csv</code>.
        * Get the next trading date after 'end_date_data'.
        * Search in <code>tickers_changes.csv</code> if there is a ticker change on this date.
        * If it does:
            * The stock data will be merged.
            * In <code>tickers_v4.csv</code> we will change "ticker" to the new ticker and add a list [date, ticker] to "tickers_old".
            * All other rows will be merged such as "start_date". For identifiers such as FIGI we will take the last available value. For the ID we will use the new ticker with the combined start_date. So this means that in our <code>processed/m1/</code> folder there will be all tickers including merged ones. So the old FB, the old META and the merged FB+META named <code>META-yyyy-mm-dd.csv</code> will be in the same folder. You could delete the old FB and META but I keep them for debugging purposes.
            * The row of the old ticker will be deleted
            * **We need to restart the loop!** If we don't the following can happen: Let's assume that a ticker was renamed from A -> B -> C -> D but that the order in which it appears in our ticker list is C, D, A, B. Using our loop, C gets merged with D. Then the loop checks D, which has no renamings. Then A gets merged with B. Then B gets merged with C, however that is incorrect! B should be merged with the new D, which contains C. Any double+ renamings have the risk of being in the 'wrong order'.

Note: if a ticker A goes OTC and then comes back and changes to B, then we will have two files: one of the A before OTC and the A+B after OTC named B.

In [1]:
from utils import get_tickers, get_market_dates, get_ticker_changes, get_market_hours
from datetime import datetime, date, time
import mplfinance as mpf
import pandas as pd
import numpy as np
import os
DATA_PATH = "../../../data/polygon/"
END_DATE = date(2023, 9, 1)

In [ ]:
tickers_v4 = get_tickers(v=4)
market_dates = get_market_dates()
ticker_changes = get_ticker_changes()

tickers_v4.insert(loc = 2, column = 'tickers_old', value = [[] for _ in range(len(tickers_v4))])

while True:
    # tickers_v4 gets smaller by 1 element every time we run this loop.
    for index_from, row_from in tickers_v4.copy().iterrows():
        # Get values
        type_from = row_from['type']
        if type_from == "INDEX":
            continue
        id_from = row_from['ID']
        ticker_from = row_from['ticker']
        start_date_from = row_from['start_date']
        end_date_from = row_from['end_date']
        start_data_from = row_from['start_data']
        end_data_from = row_from['end_data']
        if end_data_from == END_DATE:
            continue

        start_data_to = market_dates[market_dates.index(end_data_from) + 1]

        # Get ticker changes 
        change = ticker_changes[(ticker_changes.index == start_data_to) & (ticker_changes['from'] == ticker_from)]
        if change.empty:
            continue
        elif len(change) > 1:
            raise Exception("Duplicate!")
        ticker_to = change['to'].values[0]

        # Set values of new ticker
        row_to = tickers_v4[(tickers_v4['start_data'] == start_data_to) & (tickers_v4['ticker'] == ticker_to)]
        if row_to.empty:
            continue
        index_to = row_to.index[0]
        id_to = row_to['ID'].values[0]
        new_id = ticker_to + "-" + start_date_from.isoformat()
        tickers_v4.loc[index_to, "ID"] = new_id
        tickers_v4.loc[index_to, "tickers_old"].append([start_data_to.isoformat(), ticker_from])
        tickers_v4.loc[index_to, "start_date"] = start_date_from
        tickers_v4.loc[index_to, "start_data"] = start_data_from

        # Do the actual merging
        from_ = pd.read_parquet(DATA_PATH + f"processed/m1/{id_from}.parquet")
        to = pd.read_parquet(DATA_PATH + f"processed/m1/{id_to}.parquet")

        # Because companies like to be annoying, a split/dividend can take place at the same time as a ticker change. We have to account for this. An example is TYDE -> OCTO with a 50:1 reverse split. However this is much easier than 5_process_raw_data.ipynb, because there is at most a single split. This is rare, but there are still a handful of cases.
        if os.path.isfile(DATA_PATH + f"raw/adjustments/{ticker_to}.csv"):
            adjustments = pd.read_csv(DATA_PATH + f"raw/adjustments/{ticker_to}.csv", parse_dates=True, index_col=0)
            adjustments.index = pd.to_datetime(adjustments.index).date
            adjustments = adjustments[(adjustments.index == start_data_to)]

            # SPLIT ADJUSTMENT
            split = adjustments[adjustments.type == 'SPLIT']
            if len(split) > 0:
                split_amount = split['amount'][0]

                from_[['open', 'high', 'low', 'close']] = from_[['open', 'high', 'low', 'close']].multiply(split_amount)
                from_['volume'] = from_['volume'].divide(split_amount)

            # DIVIDEND ADJUSTMENT - REUN is the only case, not clear what happened there.
            dividend = adjustments[adjustments.type == 'DIV']
            if len(dividend) > 0:
                print(ticker_to)
                market_hours = get_market_hours()
                market_hours = market_hours[['regular_close']]

                cum_div_date = end_data_from
                cum_div_time = market_hours.loc[cum_div_date][0]
                cum_div_datetime = datetime.combine(cum_div_date, cum_div_time)
                cum_div_datetime = (from_[from_.index <= cum_div_datetime].index).max()
                cum_div_close = from_.loc[cum_div_datetime, 'close']
                dividend = dividend['amount'][0]
                    
                adjustment_factor = 1 - dividend/cum_div_close

                from_[['open', 'high', 'low', 'close']] = from_[['open', 'high', 'low', 'close']].multiply(adjustment_factor)
                from_['volume'] = from_['volume'].divide(adjustment_factor)
                from_ = round(from_, 4)
                from_['volume'] = from_['volume'].astype(int)\
            
            # ROUNDING
            if len(split) > 0 or len(dividend) > 0:
                from_ = round(from_, 4)
                from_['volume'] = from_['volume'].astype(int)

        # If on the old ticker, there are divs/splits on start_data_to (start of new ticker), then something is wrong.
        if os.path.isfile(DATA_PATH + f"raw/adjustments/{ticker_from}.csv"):
            adjustments = pd.read_csv(DATA_PATH + f"raw/adjustments/{ticker_from}.csv", parse_dates=True, index_col=0)
            adjustments.index = pd.to_datetime(adjustments.index).date
            adjustments = adjustments[(adjustments.index == start_data_to)]
            assert len(adjustments) == 0

        pd.concat([from_, to]).to_parquet(DATA_PATH + f"processed/m1/{new_id}.parquet", engine="pyarrow", compression = 'brotli')

        # Delete old ticker from ticker list. You can also remove the files, but that is not necessary. In backtesting you will always loop over the ticker list and not over de files in the folder.
        # os.remove(path = DATA_PATH + f"processed/m1/{id_from}.parquet") # Removal of old renamed ticker (not necessary)
        # os.remove(path = DATA_PATH + f"processed/m1/{id_to}.parquet") # Remove of new ticker (not necessary)
        tickers_v4.drop(index_from, inplace=True)
        tickers_v4.reset_index(inplace=True, drop=True)
        print(f"Ticker change {ticker_from} -> {ticker_to} on {start_data_to} has been processed")
        print(f"{index_from/len(tickers_v4)*100:.1f}% | Length of tickers_v4 is {len(tickers_v4)}")
        break
    
    # If we have reached the end of the loop, it means we have processed everything. Then we can stop.
    if index_from >= (len(tickers_v4)-1):
        break

tickers_v4.to_csv("../../../data/tickers_v5.csv")

In [3]:
tickers_v5 = get_tickers(v=5)
renamings = tickers_v5[tickers_v5["tickers_old"].str.len() > 2] # These were renamed
print(len(renamings))
tickers_v5[tickers_v5['ticker'] == 'META']

550


,ID,ticker,tickers_old,name,active,start_date,end_date,start_data,end_data,type,cik,composite_figi
3976,META-2019-01-01,META,"[['2022-06-09', 'FB']]","Meta Platforms, Inc. Class A Common Stock",True,2019-01-01,2023-09-01,2019-01-02,2023-09-01,CS,1326801.0,BBG000MM2P62


Tickers that were renamed multiple times:

In [4]:
multiple_renamings = tickers_v5[tickers_v5["tickers_old"].str.len() > 30]
print(len(multiple_renamings))
multiple_renamings.head(2)

4


,ID,ticker,tickers_old,name,active,start_date,end_date,start_data,end_data,type,cik,composite_figi
2162,ERNA-2019-01-01,ERNA,"[['2022-10-17', 'BTX'], ['2021-03-26', 'NTN']]",Eterna Therapeutics Inc. Common Stock,True,2019-01-01,2023-09-01,2019-01-02,2023-09-01,CS,748592.0,BBG000CX5677
3448,JXJT-2019-01-01,JXJT,"[['2022-10-10', 'LLL'], ['2021-10-13', 'KBSF']]",JX Luxventure Limited Common Stock,True,2019-01-01,2023-09-01,2019-01-02,2023-09-01,CS,1546383.0,


Now we have 5 tickers lists. These are:
1. Basic ticker list with a lot of incorrect duplications.
2. Duplications merged and incorrect tickers removed.
3. ETFs added.
4. Data start/end dates added.
5. Renamings merged.
Only the last should be used in backtesting.

If Polygon just provided these from the start, it would have saved countless hours. But at least I learned some Python I guess. And at least Polygon does not ask thousands.

# 9.2 Updates
Rerun the file after setting END_DATE and updating the list of renamings.